In [1]:
import os
# if using Apple MPS, fall back to CPU for unsupported ops
os.environ["PYTORCH_ENABLE_MPS_FALLBACK"] = "1"
import numpy as np
import torch
import matplotlib.pyplot as plt
from PIL import Image

In [ ]:
# select the device for computation
if torch.cuda.is_available():
    device = torch.device("cuda")
elif torch.backends.mps.is_available():
    device = torch.device("mps")
else:
    device = torch.device("cpu")
print(f"using device: {device}")

if device.type == "cuda":
    # use bfloat16 for the entire notebook
    torch.autocast("cuda", dtype=torch.bfloat16).__enter__()
    # turn on tfloat32 for Ampere GPUs (https://pytorch.org/docs/stable/notes/cuda.html#tensorfloat-32-tf32-on-ampere-devices)
    if torch.cuda.get_device_properties(0).major >= 8:
        torch.backends.cuda.matmul.allow_tf32 = True
        torch.backends.cudnn.allow_tf32 = True
elif device.type == "mps":
    print(
        "\nSupport for MPS devices is preliminary. SAM 2 is trained with CUDA and might "
        "give numerically different outputs and sometimes degraded performance on MPS. "
        "See e.g. https://github.com/pytorch/pytorch/issues/84936 for a discussion."
    )

In [3]:
np.random.seed(3)

def show_mask(mask, ax, random_color=False, borders = True):
    if random_color:
        color = np.concatenate([np.random.random(3), np.array([0.6])], axis=0)
    else:
        color = np.array([30/255, 144/255, 255/255, 0.6])
    h, w = mask.shape[-2:]
    mask = mask.astype(np.uint8)
    mask_image =  mask.reshape(h, w, 1) * color.reshape(1, 1, -1)
    if borders:
        import cv2
        contours, _ = cv2.findContours(mask,cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE) 
        # Try to smooth contours
        contours = [cv2.approxPolyDP(contour, epsilon=0.01, closed=True) for contour in contours]
        mask_image = cv2.drawContours(mask_image, contours, -1, (1, 1, 1, 0.5), thickness=2) 
    ax.imshow(mask_image)

def show_points(coords, labels, ax, marker_size=375):
    pos_points = coords[labels==1]
    neg_points = coords[labels==0]
    ax.scatter(pos_points[:, 0], pos_points[:, 1], color='green', marker='*', s=marker_size, edgecolor='white', linewidth=1.25)
    ax.scatter(neg_points[:, 0], neg_points[:, 1], color='red', marker='*', s=marker_size, edgecolor='white', linewidth=1.25)   

def show_box(box, ax):
    x0, y0 = box[0], box[1]
    w, h = box[2] - box[0], box[3] - box[1]
    ax.add_patch(plt.Rectangle((x0, y0), w, h, edgecolor='green', facecolor=(0, 0, 0, 0), lw=2))    

def show_masks(image, masks, scores, point_coords=None, box_coords=None, input_labels=None, borders=True):
    for i, (mask, score) in enumerate(zip(masks, scores)):
        plt.figure(figsize=(10, 10))
        plt.imshow(image)
        show_mask(mask, plt.gca(), borders=borders)
        if point_coords is not None:
            assert input_labels is not None
            show_points(point_coords, input_labels, plt.gca())
        if box_coords is not None:
            # boxes
            show_box(box_coords, plt.gca())
        if len(scores) > 1:
            plt.title(f"Mask {i+1}, Score: {score:.3f}", fontsize=18)
        plt.axis('off')
        plt.show()

In [4]:
image = Image.open('E:\\PycharmProjects\\yolov8_datasets\\ChuangDian\\20250108_plastic_bottle\\0111\\updated_json\\6.bmp')
image = np.array(image.convert("RGB"))

In [ ]:
plt.figure(figsize=(10, 10))
plt.imshow(image)
plt.axis('on')
plt.show()

In [ ]:
from sam2.build_sam import build_sam2
from sam2.sam2_image_predictor import SAM2ImagePredictor

sam2_checkpoint = "../checkpoints/sam2.1_hiera_large.pt"
model_cfg = "configs/sam2.1/sam2.1_hiera_l.yaml"

sam2_model = build_sam2(model_cfg, sam2_checkpoint, device=device)

predictor = SAM2ImagePredictor(sam2_model)

In [7]:
predictor.set_image(image)

In [ ]:
import sys
import os
# 获取当前 Notebook 文件的目录路径
current_dir = os.path.dirname(os.path.abspath("__file__"))
#假设 switch nerf 目录在项目根目录下，获取项目根目录
project_root = os.path.abspath(os.path.join(current_dir, ".."))
#将项目根目录添加到 sys.path 中
if project_root not in sys.path:
    sys.path.append(project_root)
#打印路径和目录内容以确保正确性
print("current directory:",current_dir)
print("project root directory:",project_root)
print("sys.path:",sys.path)

from utils_pan import get_box_from_labelme_json, load_json, update_json_with_polygon, save_json
json_path = 'E:\\PycharmProjects\\yolov8_datasets\\ChuangDian\\20250108_plastic_bottle\\0111\\updated_json\\6_1.json'
label = 'pingti'
input_box = get_box_from_labelme_json(json_path=json_path, 
                                      label_name=label, shape_type='polygon')

In [9]:
masks, scores, _ = predictor.predict(
    point_coords=None,
    point_labels=None,
    box=input_box[None, :],
    multimask_output=False,
)

In [ ]:
show_masks(image, masks, scores, box_coords=input_box)

In [ ]:
import numpy as np
import cv2
import matplotlib.pyplot as plt

# 转换为 8 位无符号整数类型
mask = masks[0].astype(np.uint8) * 255  # 将前景变为255，背景为0

# 查找轮廓
contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

# 选择最大的轮廓作为前景的边缘
# 如果有多个轮廓，可以选择面积最大的轮廓作为主体
max_contour = max(contours, key=cv2.contourArea)

# 获取该轮廓的多边形逼近
epsilon = 0.001 * cv2.arcLength(max_contour, True)  # 误差范围，可以调整
polygon = cv2.approxPolyDP(max_contour, epsilon, True)

# 显示结果
# 创建一个与 mask 大小相同的黑色图像，用于显示多边形
result_image = np.zeros_like(mask)
# print(polygon)
# print(len(polygon))

# 将多边形绘制到结果图像上
cv2.polylines(result_image, [polygon], isClosed=True, color=(255), thickness=2)

# 使用 matplotlib 显示结果
plt.imshow(result_image, cmap='gray')
plt.title("Polygon Approximation of Foreground")
plt.show()

In [12]:
# 加载现有的 JSON 数据
json_data = load_json(json_path)
update_json_with_polygon(json_data, label, polygon)

# 保存更新后的 JSON 文件
save_json(json_data, json_path)
